In [ ]:
from nsepy import get_history
import pandas as pd   
import numpy as np 
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from datetime import date 
import yfinance as yf 

In [ ]:
pharma_list = ["DIVISLAB.NS", "DRREDDY.NS", "CIPLA.NS", "SUNPHARMA.NS", "AUROPHARMA.NS","ABBOTINDIA.NS", 
               "GLAND.NS","LAURUSLABS.NS","GRANULES.NS", "ALKEM.NS","IPCALAB.NS","AJANTPHARM.NS","LUPIN.NS",
              "TORNTPHARM.NS", "CADILAHC.NS","GLAXO.NS","PFIZER.NS","SANOFI.NS","APLLTD.NS", "NATCOPHARM.NS",
              "GLENMARK.NS", "SUVENPHAR.NS","ERIS.NS","AARTIDRUGS.NS","WOCKPHARMA.NS"] 
pharma_data= {}

In [ ]:
for ticker in pharma_list: 
    ticker_object = yf.Ticker(ticker) 

    #convert info() output from dictionary to dataframe
    temp = pd.DataFrame.from_dict(ticker_object.info, orient="index") 
    temp.reset_index(inplace=True)
    temp.columns = ["Attribute", "Recent"]
    
    # add (ticker, dataframe) to main dictionary
    pharma_data[ticker] = temp
pharma_data

In [ ]:
combined_data = pd.concat(pharma_data)
combined_data = combined_data.reset_index() 

In [ ]:
del combined_data["level_1"]
combined_data.columns = ["Ticker", "Attribute", "Recent"] # update column names 

combined_data

In [ ]:
enterpriseToEbitda = combined_data[combined_data["Attribute"]=="enterpriseToEbitda"].reset_index() 
del enterpriseToEbitda["index"] # clean up unnecessary column 

In [ ]:
enterpriseToEbitda_sorted = enterpriseToEbitda.sort_values('Recent',ascending=True) 
enterpriseToEbitda_sorted

In [ ]:
enterpriseToEbitda = combined_data[combined_data["Attribute"]=="enterpriseToEbitda"].reset_index()
forwardPE=combined_data[combined_data["Attribute"]=="forwardPE"].reset_index()
trailingPE=combined_data[combined_data["Attribute"]=="trailingPE"].reset_index()
earninggrowth=combined_data[combined_data["Attribute"]=="earningsGrowth"].reset_index() 

del enterpriseToEbitda["index"] # clean up unnecessary column
del forwardPE["index"]
del trailingPE["index"]

pharma_valuation1=enterpriseToEbitda.join(forwardPE['Recent'],lsuffix='_ev/ebitda',rsuffix='_forwardPE') 
del pharma_valuation1['Attribute'] 
pharma_valuation1 
pharma_valuation2=pharma_valuation1.join(trailingPE['Recent'],lsuffix='',rsuffix='') 
pharma_valuation2=pharma_valuation2.rename(columns={'Recent':'TrailingPE'})  
pharma_valuation3=pharma_valuation2.join(earninggrowth['Recent'],lsuffix='',rsuffix='') 
pharma_valuation3=pharma_valuation3.rename(columns={'Recent':'Earninggrowth'})  
pharma_valuation3

In [ ]:
pharma_valuation=pharma_valuation3[0:24]
x = pharma_valuation.iloc[:, [1, 4]].values  

In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=3, n_init=5)
kmeans.fit_predict(x)

In [ ]:
pharma_valuation['Cluster']=kmeans.fit_predict(x)

In [ ]:
pharma_valuation

In [ ]:
pharma_valuation['Cluster']=pharma_valuation['Cluster'].astype("category")

In [1]:
sns.relplot(x="Recent_ev/ebitda",y="Earninggrowth",hue='Cluster',data=pharma_valuation, height=6)

NameError: name 'sns' is not defined